<a href="https://colab.research.google.com/github/kengustafson/DeepDive-DataScience/blob/main/Project_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mastery (er, Fundamentals) of SQL**

**Deep Dive Data Science Project 5**

---
Ken Gustafson 4/6/2021

## Problem Definition

We desire to explore open datasets for possible insight.

Output to be produced: Insights and some useful plots

## Data Collection

In [ ]:
# Here we import the necessary packages

from google.cloud import bigquery
from google.colab import auth
import pandas as pd
import plotly.express as px

auth.authenticate_user()

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image 

Additional criteria:

In [ ]:
# Data was sourced from Google BiqQuery open data sets.
# Project Settings

project_id = 'ddds-project5'
client = bigquery.Client(project=project_id)

project_dataset = client.dataset('traffic_safety', project = 'dataflix-public-datasets')
traffic_safety = client.get_dataset(project_dataset)

## Initial Exploration

Data 360 Dataset - https://console.cloud.google.com/marketplace/product/dataflix-public-datasets/data-360-traffic-and-safety

Dataset documentation: https://www.dataflix.com/data360/docs/

In [ ]:
# Examine available tables

tables = list(client.list_tables(traffic_safety))
print([table.table_id for table in tables])

['accident', 'cevent', 'damage', 'distract', 'drimpair', 'drugs', 'factor', 'maneuver', 'nmcrash', 'nmimpair', 'nmprior', 'person', 'safetyeq', 'vehicle', 'vevent', 'violatn', 'vision', 'vsoe']


In [ ]:
# Get table reference for accidents, check schema

table_ref = traffic_safety.table('accident')
accident = client.get_table(table_ref)
accident.schema

[SchemaField('L_YEAR', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('STATE', 'STRING', 'NULLABLE', None, ()),
 SchemaField('ST_CASE', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('VE_TOTAL', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('VE_FORMS', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('PVH_INVL', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('PEDS', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('PERNOTMVIT', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('PERMVIT', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('PERSONS', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('COUNTY', 'STRING', 'NULLABLE', None, ()),
 SchemaField('CITY', 'STRING', 'NULLABLE', None, ()),
 SchemaField('DAY', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('MONTH', 'STRING', 'NULLABLE', None, ()),
 SchemaField('YEAR', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('WEEK_DAY', 'STRING', 'NULLABLE', None, ()),
 SchemaField('HOUR', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('MINUTE', 'INTEGER', 

## Exploratory Data Analysis

In [ ]:
dataset = 'dataflix-public-datasets.traffic_safety'

In [ ]:
# Check a few entries in Accident table

query = f""" 
        SELECT *
        FROM `{dataset}.accident`
        WHERE L_YEAR = 2018 OR L_YEAR = 2017
        LIMIT 5
        """

df = client.query(query).to_dataframe()
df

,L_YEAR,STATE,ST_CASE,VE_TOTAL,VE_FORMS,PVH_INVL,PEDS,PERNOTMVIT,PERMVIT,PERSONS,COUNTY,CITY,DAY,MONTH,YEAR,WEEK_DAY,HOUR,MINUTE,NHS,RUR_URB,FUNC_SYS,RD_OWNER,ROUTE,TWAY_ID,TWAY_ID2,MILEPT,LATITUDE,LONGITUD,SP_JUR,HARM_EV,MAN_COLL,RELJCT1,RELJCT2,TYP_INT,WRK_ZONE,REL_ROAD,LGT_COND,WEATHER1,WEATHER2,WEATHER,SCH_BUS,RAIL,NOT_HOUR,NOT_MIN,ARR_HOUR,ARR_MIN,HOSP_HR,HOSP_MN,CF1,CF2,CF3,FATALS,DRUNK_DR,DATE_PARTITION
0,2017,North Carolina,370899,1,1,0,5,5,1,1,HAYWOOD (87),None,1,September,2017,Friday,12,32,This section IS ON the NHS,RURAL,"Principal arterial, other",State Highway Agency,U.S. Highway,US-276,None,0,35.516708,-83.087003,No Special Jurisdiction (Includes National For...,Ridden Animal or Animal Drawn Conveyance,Not a Collision with Motor Vehicle in Transport,No,Non-Junction,Not an Intersection,None,On Roadway,Daylight,Rain,Cloudy,Rain,No,0,12,30,12,37,13,28,None,None,None,1,0,2017-01-01
1,2017,Virginia,510061,1,1,0,3,3,1,1,BLAND (21),None,20,February,2017,Monday,8,30,This section IS NOT on the NHS,RURAL,Collector,State Highway Agency,County Road,CR-606,None,113,37.177639,-80.928231,No Special Jurisdiction (Includes National For...,Pedestrian,Not a Collision with Motor Vehicle in Transport,No,Non-Junction,Not an Intersection,None,On Roadside,Daylight,"Fog, Smog, Smoke",No Additional Atmospheric Conditions,"Fog, Smog, Smoke",No,0,99,99,99,99,88,88,None,None,None,2,0,2017-01-01
2,2017,Texas,481854,1,1,0,5,5,1,1,MEDINA (325),None,29,July,2017,Saturday,2,20,This section IS ON the NHS,RURAL,"Interstate, principal arterial",State Highway Agency,Local Street - Frontage Road,I-35,None,125,29.213300,-98.805761,No Special Jurisdiction (Includes National For...,Pedestrian,Not a Collision with Motor Vehicle in Transport,No,Non-Junction,Not an Intersection,None,On Roadway,Dark - Unknown Lighting,Clear,No Additional Atmospheric Conditions,Clear,No,0,99,99,99,99,99,99,Recent/Previous accident scene nearby,None,None,3,0,2017-01-01
3,2017,Indiana,180496,1,1,0,7,7,1,1,ADAMS (1),GENEVA,30,July,2017,Sunday,10,15,This section IS ON the NHS,RURAL,"Principal arterial, other",State Highway Agency,U.S. Highway,US-27S,None,99998,40.570467,-84.959086,No Special Jurisdiction (Includes National For...,Ridden Animal or Animal Drawn Conveyance,Not a Collision with Motor Vehicle in Transport,No,Non-Junction,Not an Intersection,None,On Roadway,Daylight,Clear,No Additional Atmospheric Conditions,Clear,No,0,99,99,99,99,99,99,None,None,None,1,0,2017-01-01
4,2017,Mississippi,280317,1,1,0,3,3,1,1,LAMAR (73),SUMRALL,15,March,2017,Wednesday,18,18,This section IS NOT on the NHS,RURAL,Collector,City or Municipal Highway Agency,Local Street - Municipality,OLOH RD,None,0,31.394264,-89.542253,No Special Jurisdiction (Includes National For...,Pedestrian,Not a Collision with Motor Vehicle in Transport,Yes,Not Defined,Not an Intersection,None,Off Roadway-Location Unknown,Daylight,Clear,No Additional Atmospheric Conditions,Clear,No,0,99,99,99,99,99,99,None,None,None,2,1,2017-01-01


Examine how many Accident records there are

In [ ]:
query = f"""
        SELECT COUNT(*)
        FROM `{dataset}.accident`
        """

df = client.query(query).to_dataframe()
df.head()

,f0_
0,1666447


What time/years are included...


In [ ]:
query = f"""
        SELECT DISTINCT YEAR
        FROM `{dataset}.accident`
        ORDER BY YEAR DESC
        """

df = client.query(query).to_dataframe()
print(df['YEAR'].to_list())

[2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, 1979, 1978, 1977, 1976, 1975, 97, 96, 95, 94, 93, 92, 91, 90, 88, 87, 86, 85, 84, 83, 82, 81, 80, 78]


There are two entry types for YEAR, 2-digits (90) and 4-digits (1990), so any YEAR query may have to account for this.

Let's see data for New Mexico

In [ ]:
query = f"""
        SELECT COUNT(*) AS Num_of_accidents, YEAR AS Year
        FROM `{dataset}.accident`
        WHERE STATE = 'New Mexico'
        GROUP BY YEAR
        ORDER BY YEAR ASC
        """

accidents_per_year = client.query(query).to_dataframe()

Turns out BigQuery is smart enough to aggregate 2-digit years with 4-digit years

In [ ]:
accidents_per_year.head()

,Num_of_accidents,Year
0,466,1975
1,461,1976
2,567,1977
3,588,1978
4,555,1979


Which month are the most accidents in?

In [ ]:
query = f"""
        SELECT COUNT(*) AS Cnt, MONTH
        FROM `{dataset}.accident`
        GROUP BY MONTH
        ORDER BY Cnt DESC
        LIMIT 1
        """

month_most = client.query(query).to_dataframe()
month_most.head()

,Cnt,MONTH
0,157789,August


Where was the southern most accident?

In [ ]:
query = f"""
        SELECT STATE, LATITUDE, LONGITUD   # HA, LONGITUD
        FROM `{dataset}.accident`
        WHERE LATITUDE = (
            SELECT MIN(LATITUDE)
            FROM `{dataset}.accident`)
        """

southern_most = client.query(query).to_dataframe()
southern_most

,STATE,LATITUDE,LONGITUD
0,Puerto Rico,18.375736,-66.147717


How many vehicles have been involved in accidents in New Mexico?

In [ ]:
query = f"""
        SELECT SUM(VE_TOTAL) AS Total_Veh, STATE
        FROM `{dataset}.accident`
        WHERE STATE = 'New Mexico'
        GROUP BY STATE
        """

total_veh = client.query(query).to_dataframe()
total_veh

,Total_Veh,STATE
0,6754,New Mexico


### Plots

Let's see how the number of accidents in New Mexico has changed over the years.

In [ ]:
fig = px.line(accidents_per_year, y ='Num_of_accidents', x='Year')
fig.show()

How has the average number of fatalities per accident for Bernalillo County changed?

In [ ]:
query = f"""
        SELECT YEAR, AVG(FATALS) AS Avg_Fatals_per_Accident
        FROM `{dataset}.accident`
        WHERE STATE LIKE '% Mexico' AND COUNTY = 'BERNALILLO (1)'
        GROUP BY YEAR
        ORDER BY YEAR ASC
        """

bern_fat_per_year = client.query(query).to_dataframe()

In [ ]:
fig = px.line(bern_fat_per_year, y ='Avg_Fatals_per_Accident', x='YEAR')
fig.show()

What accident involved the most vehicles?

In [ ]:
query = f"""
        SELECT VE_TOTAL, STATE, YEAR, MONTH, DAY, HOUR
        FROM `{dataset}.accident`
        WHERE VE_TOTAL = (
            SELECT MAX(VE_TOTAL)
            FROM `{dataset}.accident`)
        """

most_vehicles = client.query(query).to_dataframe()
most_vehicles

,VE_TOTAL,STATE,YEAR,MONTH,DAY,HOUR
0,92,California,2007,November,3,7


https://www.cbsnews.com/news/100-car-pileup-kills-2-in-california/

Which accident had the most fatalities?

In [ ]:
query = f"""
        SELECT FATALS, STATE, YEAR, MONTH, DAY, HOUR
        FROM `{dataset}.accident`
        WHERE FATALS = (
            SELECT MAX(FATALS)
            FROM `{dataset}.accident`)
        """

most_fatalities = client.query(query).to_dataframe()
most_fatalities

,FATALS,STATE,YEAR,MONTH,DAY,HOUR
0,29,California,1976,May,21,10


https://en.wikipedia.org/wiki/Yuba_City_bus_disaster

How many bicyclists were in accidents, per state, over the years 2017 and 2018?

In [ ]:
query = f"""
        SELECT a.STATE, COUNT(*) AS Bicyclists
        FROM `{dataset}.accident` AS a
        INNER JOIN `{dataset}.person` AS p
        ON a.STATE = p.STATE AND a.ST_CASE = p.ST_CASE AND a.L_YEAR = p.L_YEAR
        WHERE p.PER_TYP = 'Bicyclist' AND a.L_YEAR BETWEEN 2017 AND 2018
        GROUP BY a.STATE
        ORDER BY Bicyclists DESC
        """

bicy_per_state = client.query(query).to_dataframe()
bicy_per_state

,STATE,Bicyclists
0,Florida,297
1,California,286
2,Texas,133
3,New York,78
4,Arizona,54
5,Louisiana,51
6,Illinois,51
7,North Carolina,47
8,Georgia,46
9,Michigan,46


In [ ]:
fig = px.bar(bicy_per_state, x='STATE', y='Bicyclists', color='Bicyclists',
             labels={'Bicyclists':'Bicyclist-involved Accidents'})
fig.show()

## Conclusions

- SQL is appropriate for retrieving data from an RDB
- Ample open datasets to chose from
- Different insights in each table
- Accidents suck
- Plotly Express is AWESOME!